## jVS_HIHOで付けたJVSの音素アライメントが、音声と一致しているか確認する


## 準備

### ライブラリのインポート

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import librosa
import librosa.display
from scipy import ndimage
from IPython.display import Audio
from dotenv import load_dotenv
import os

### データの読み込み

In [ ]:
load_dotenv()

data_dir = os.getenv("DATA_DIR")
jvs_hiho_dir = os.getenv("JVS_HIHO_DIR")

jvs_dir = Path(data_dir) / "jvs_takeshun_ver1"
jvs_alignment_dir = Path(jvs_hiho_dir) / "aligned_labels_julius_takehun"
jvs_transcirpt_path = Path(jvs_hiho_dir) / "voiceactoress100_spaced_julius.txt"

print(f"jvs_dir: {jvs_dir}")
print(f"jvs_alignment_dir: {jvs_alignment_dir}")
print(f"jvs_transcirpt_path: {jvs_transcirpt_path}")
assert jvs_dir.exists()
assert jvs_alignment_dir.exists()
assert jvs_transcirpt_path.exists()

In [ ]:
jvs_dir.resolve()

In [ ]:
wav_files = jvs_dir.glob("*/parallel100/*/*.wav")
list(wav_files)[0].parts[-4]

In [ ]:
int("002")

In [ ]:
# 対応するファイルをデータフレームに読み込む
df = []

wav_files = jvs_dir.glob("*/parallel100/*/*.wav")

with open(jvs_transcirpt_path, "r") as f:
    # １行ずつのリストにする。
    lines = f.readlines()

if len(lines) != 100:
    print(f"lines length is {len(lines)}.")

for wav_path in wav_files:
    wav_path_parts = wav_path.parts
    spk_id, _, _, wav_name = wav_path_parts[-4:]
    wav_stem = Path(wav_name).stem

    lab_path = jvs_alignment_dir / spk_id / wav_name.replace(".wav", ".lab")
    if not lab_path.exists():
        print(f"{lab_path} does not exist.")

    wav_id = int(wav_stem.split("_")[-1])  # 1~100
    transcript = lines[wav_id - 1]

    df.append(
        {
            "spk_id": spk_id,
            "wav_id": wav_id,
            "wav_stem": wav_stem,
            "transcript": transcript,
            "wav_path": wav_path,
            "lab_path": lab_path,
        }
    )

df = pd.DataFrame(df)
display(df.info())
display(df.head())

In [ ]:
# debug
df_tmp = df.iloc[:10]
df_tmp.info()

## 可視化

### 便利な抽出関数

In [ ]:
# 音声波形抽出
# wav_path -> wav, sr
def extract_waveform(audio_file_path, sr=24000):
    waveform, sample_rate = librosa.load(audio_file_path, mono=True)
    return waveform, sample_rate


# wav -> db変換
def convert_db(waveform):
    db = librosa.power_to_db(waveform)
    return db


# 連続区間抽出
# db -> bool_list
def run_length_encoding(arr, min_run_length=3):
    diff = np.diff(arr)  # 隣接要素の差分を計算
    run_starts = np.where(diff != 0)[0] + 1  # 差分が0でないインデックスを取得し、連続する範囲の開始位置を得る
    run_lengths = np.diff(np.concatenate(([0], run_starts, [len(arr)])))  # 連続する範囲の長さを計算
    result = np.repeat(run_lengths >= min_run_length, run_lengths)  # 連続する範囲をTrueに変換
    return result


# Pause区間抽出
# db, db_threshold, time_threshold, sr -> pause_bool_list
# 閾値を超えたらpauseとみなす
def detect_pause_position(
    db_sequence, db_threshold=-50, time_threshold=50 / 1000, sample_rate=24000
):
    """dbと音声長の閾値からpauseの位置を判定する。

    Args:
        db_sequence (np.array): 音声波形をdbに変換した配列
        db_threshold (float): 無音区間とするdbの閾値
        time_threshold (float): 無音区間が連続した時にpauseとみなす時間の閾値

    Returns:
        pause_positions (list): pauseの位置のリスト
    """
    under_db_threshold = db_sequence < db_threshold

    # 連続区間を抽出
    sample_threshold = int(time_threshold * sample_rate)
    is_continuous = run_length_encoding(under_db_threshold, sample_threshold)

    # pauseの位置を抽出
    pause_positions = under_db_threshold & is_continuous

    return pause_positions


# pause区間付きの波形の可視化
def plot_db_with_pause(db, sr, db_threshold, time_threshold, xlim=None):
    fig, ax = plt.subplots(figsize=(20, 5))
    x = np.arange(len(db)) / sr
    ax.plot(x, db, label="db")

    # dbの閾値を引く
    ax.axhline(
        y=db_threshold,
        color="r",
        linestyle="-",
        linewidth=2,
        alpha=0.7,
        label="db_threshold",
    )

    # pauseの領域を塗りつぶす
    pause_position = detect_pause_position(db, db_threshold, time_threshold, sr)
    plt.fill_between(x, -80, 0, where=pause_position, facecolor="b", alpha=0.5)

    ax.set_xlim(xlim)
    ax.legend()
    plt.show()


# 波形の可視化
def plot_wavform(waveform, sr, xlim=None):
    fig, ax = plt.subplots(figsize=(20, 5))
    x = np.arange(len(waveform)) / sr
    ax.plot(x, waveform, label="waveform")
    ax.set_xlim(xlim)
    ax.legend()
    plt.show()


# 音声再生ボタン生成
def play_button(waveform, sr):
    display(Audio(waveform, rate=sr, autoplay=True))


# アライメントの抽出
# lab_path -> df_lab
def read_lab(lab_path):
    """labファイルを読み込む"""
    # labファイルがない場合
    if not Path(lab_path).exists():
        print(f"{lab_path} does not exist.")
        return None

    # labファイルがある場合
    df_lab = []
    with open(lab_path, "r") as f:
        for phoneme_idx, line in enumerate(f):
            if line == "":
                continue
            start, end, phoneme = line.split()
            duration = float(end) - float(start)
            df_lab.append(
                {
                    "start": float(start),
                    "end": float(end),
                    "phoneme": phoneme,
                    "phoneme_idx": phoneme_idx,
                    "duration": duration,
                }
            )
    df_lab = pd.DataFrame(df_lab)
    return df_lab


# アライメントの可視化
def plot_phoneme_alignment(lab_path, xlim=None):
    """Labファイルから音素のアライメントをプロットする

    Args:
        lab_path (_type_): Labファイルのパス
    """
    df = read_lab(lab_path)
    display(df[-10:])

    # 描画
    fig, ax = plt.subplots(figsize=(20, 2))
    for start, end, label, _, _ in df.values:
        ax.axvline(start, color="gray", linestyle="--")
        ax.axvline(end, color="gray", linestyle="--")
        # ax.text((start + end) / 2, 0.5, label, ha='center', va='bottom', fontsize=20)
        # ax.text(start + (end-start), 0.5, label, ha='center', va='bottom', fontsize=20)
    # ax.set_yticks([])
    ax.set_xlim(xlim)
    ax.set_xlabel("Time (seconds)")
    fig.tight_layout()
    plt.legend()
    plt.show()


# 並べて可視化する。
def plot_all(
    df_tmp_iloc, sample_rate=24000, db_threshold=-50, time_threshold=50 / 1000
):
    spk_id, wav_id, transcript, wav_path, lab_path = df_tmp_iloc[
        "spk_id wav_id transcript wav_path lab_path".split(" ")
    ]
    wav, sr = extract_waveform(wav_path, sr=sample_rate)
    db = convert_db(wav)
    xlim = (0, len(wav) / sr)

    fig, ax = plt.subplots(
        3, 1, figsize=(20, 10), gridspec_kw={"height_ratios": [4, 4, 2]}
    )
    print("spk_id:", spk_id)
    print("wav_id:", wav_id)
    print("xlim:", xlim)
    print("transcript:", transcript)
    print("start ploting...")

    # 波形の可視化
    x = np.arange(len(wav)) / sr
    ax[0].plot(x, wav, label="waveform")
    ax[0].set_xlim(xlim)
    ax[0].legend()

    # dbの可視化
    x = np.arange(len(db)) / sr
    ax[1].plot(x, db, label="db")
    # dbの閾値を引く
    ax[1].axhline(
        y=db_threshold,
        color="r",
        linestyle="-",
        linewidth=2,
        alpha=0.7,
        label="db_threshold",
    )
    # pauseの領域を塗りつぶす
    pause_position = detect_pause_position(db, db_threshold, time_threshold, sr)
    ax[1].fill_between(x, -80, 0, where=pause_position, facecolor="b", alpha=0.5)
    ax[1].set_xlim(xlim)
    ax[1].legend()

    # アライメントの可視化
    df = read_lab(lab_path)
    # 描画
    for start, end, label, _, _ in df.values:
        ax[2].axvline(start, color="gray", linestyle="--")
        ax[2].axvline(end, color="gray", linestyle="--")
        # ax.text((start + end) / 2, 0.5, label, ha='center', va='bottom', fontsize=20)
        # ax.text(start + (end-start), 0.5, label, ha='center', va='bottom', fontsize=20)
    # ax.set_yticks([])
    ax[2].set_xlim(xlim)
    ax[2].set_xlabel("Time (seconds)")
    # ax[2].tight_layout()
    ax[2].legend()

    plt.show()

    print("done.")


def classfy_pause(
    db_sequence, lab_path, sample_rate=24000, db_threshold=-50, time_threshold=0.05
):
    """ポーズを分類する

    Args:
        df_jvs (_type_): _description_
    """
    # db_threshold = -50
    # time_threshold = 0.05
    # sample_rate = 24000

    # db_sequence = df_jvs.iloc[0]['db_sequence']
    pause_position = detect_pause_position(
        db_sequence, db_threshold, time_threshold, sample_rate
    )

    def run_length_encoding_range(arr, min_run_length=3):
        """
        Run-Length Encoding (RLE)を実行して連続している部分をTrueとしたブール配列を返す関数

        Parameters:
            arr (numpy.ndarray): 連続している部分を判定したい1次元のNumPy配列
            min_run_length (int): 連続していると判定する最小の長さ（デフォルトは3）

        Returns:
            numpy.ndarray: 連続している部分がTrueとなったブール配列
            list: 連続している部分の始点と終点のリスト [(start1, end1), (start2, end2), ...]
        """
        diff = np.diff(arr)  # 隣接要素の差分を計算
        run_starts = np.where(diff != 0)[0] + 1  # 差分が0でないインデックスを取得し、連続する範囲の開始位置を得る

        starts = np.concatenate(([0], run_starts))
        ends = np.concatenate((run_starts, [len(arr)]))
        lengths = ends - starts
        ranges = list(zip(starts, ends, lengths))

        # min_run_length以下の範囲を削除, Trueが連続しているもののみを取り出す
        ranges = [r for r in ranges if (r[2] >= min_run_length and arr[r[0]])]

        return ranges

    sample_threshold = int(time_threshold * sample_rate)
    pause_ranges = run_length_encoding_range(pause_position, sample_threshold)

    # print(pause_ranges)

    # df_lab = read_lab(df_jvs.iloc[0]['lab_path'])
    df_lab = read_lab(lab_path)

    ans = []
    for pause_range in pause_ranges:
        # df_labのstartもしくは、endが、start, endの範囲内にあるかどうか
        pause_start = pause_range[0]
        pause_end = pause_range[1]
        phoneme_start = df_lab["start"].values * sample_rate
        phoneme_end = df_lab["end"].values * sample_rate
        is_start_include = (pause_start <= phoneme_start) & (phoneme_start <= pause_end)
        is_end_include = (pause_start <= phoneme_end) & (phoneme_end <= pause_end)

        include_phonemes = df_lab[is_start_include | is_end_include]["phoneme"].values
        print(include_phonemes)
        if "silE" in include_phonemes:
            pause_type = "silE"
        elif "silB" in include_phonemes:
            pause_type = "silB"
        elif "sil" in include_phonemes:
            pause_type = "sil"
        elif "pau" in include_phonemes:
            pause_type = "pau"
        elif "sp" in include_phonemes:
            pause_type = "sp"
        else:
            pause_type = "RP"

        ans.append([pause_range[0], pause_range[1], pause_range[2], pause_type])
    return ans

### 可視化してみる

In [ ]:
db_threshold = -25  # dB
time_threshold = 50 / 1000  # 50ms
sample_rate = 24000

In [ ]:
df_tmp.columns

#### それぞれ可視化


In [ ]:
spk_id, wav_id, transcript, wav_path, lab_path = df[df["spk_id"] == "jvs001"].iloc[1][
    "spk_id wav_id transcript wav_path lab_path".split(" ")
]

wav, sr = extract_waveform(wav_path, sr=sample_rate)
db = convert_db(wav)
xlim = (0, len(wav) / sample_rate)

print("spk_id:", spk_id)
print("wav_id:", wav_id)
print("xlim:", xlim)
print("transcript:", transcript)
print("start ploting...")
plot_wavform(wav, sr=sample_rate, xlim=xlim)
plot_db_with_pause(
    db,
    sr=sample_rate,
    db_threshold=db_threshold,
    time_threshold=time_threshold,
    xlim=xlim,
)
plot_phoneme_alignment(lab_path, xlim=xlim)
play_button(wav, sr=sample_rate)
print("done.")

アライメントの文字を削除して、yticksを表示することで、x軸の縮尺が会うようにした。

#### まとめて可視化

In [ ]:
plot_all(
    df[df["spk_id"] == "jvs001"].iloc[1],
    sample_rate=sample_rate,
    db_threshold=db_threshold,
    time_threshold=time_threshold,
)

X軸の縮尺が会うようになったが、JVS_HIHOの影響か、dbが合わなくなった。  
-> 波形が大体1/2になってそうなので、db_thresholdも半分にしてみる db_threshold=-25


-> データの型を見る int float (librosaでの撮り方)

## ポーズの分析

In [ ]:
df[df["spk_id"] == "jvs001"].iloc[1]

In [ ]:
spk_id, wav_id, transcript, wav_path, lab_path = df[df["spk_id"] == "jvs001"].iloc[1][
    "spk_id wav_id transcript wav_path lab_path".split(" ")
]

wav, sr = extract_waveform(wav_path, sr=sample_rate)
db = convert_db(wav)

classfy_pause(
    db,
    lab_path,
    sample_rate=sample_rate,
    db_threshold=db_threshold,
    time_threshold=time_threshold,
)

あってないな...

In [ ]:
spk_id, wav_id, transcript, wav_path, lab_path = df[df["spk_id"] == "jvs001"].iloc[1][
    "spk_id wav_id transcript wav_path lab_path".split(" ")
]

wav, sr = extract_waveform(wav_path, sr=sample_rate)
db = convert_db(wav)
print("len", len(db) / sample_rate)

pause_bool_list = detect_pause_position(
    db,
    db_threshold=db_threshold,
    time_threshold=time_threshold,
    sample_rate=sample_rate,
)
pause_bool_list = np.array(pause_bool_list)

df_lab = read_lab(lab_path)
df_lab

#### 現状

JVS_HIHOの方法で生成したアライメント結果を使うことで、JVSの音声と、アライメントの対応を取ることができた。   
- JVS_HIHOでは、JuliusをPythonから呼び出すような方法で、アライメントを生成している。->少し結果が異なるのかも
- ただし、音声波形の絶対値が異なっており、dbの閾値が-50ではなく、-30を使うことにした。   
閾値で抽出したポーズの分類関数がうまく実装できない。

### 各話者ごとのポーズの長さの平均値と、音声の長さで割った値の平均値を計算する

labファイルから、spのdurationの合計と、silBのendとsilEのstartの差を計算する。

In [ ]:
pause_length = []
speech_length = []
for i, row in df.iterrows():
    lab_path = row["lab_path"]
    df_lab = read_lab(lab_path)
    # silBのend ~ silEのstartまでの長さ
    speech_len = df_lab.iloc[-1]["end"] - df_lab.iloc[0]["start"]
    speech_length.append(speech_len)
    # spのdurationの合計
    pause_len = df_lab[df_lab["phoneme"] == "sp"]["duration"].sum()
    pause_length.append(pause_len)
df["pause_length"] = pause_length
df["speech_length"] = speech_length

In [ ]:
df.head()

In [ ]:
# 話者ごとのポーズの長さの平均の分布
plt.figure(figsize=(16, 8))
df.groupby("spk_id")["pause_length"].mean().hist(bins=30, figsize=(16, 8))
plt.title("Distribution of mean pause length by speaker")
plt.xlabel("Mean pause length")
plt.ylabel("Number of speakers")
plt.show()

In [ ]:
# 話者ごとの音声のうち、ポーズの占める割合の平均の分布
plt.figure(figsize=(16, 8))
df_pause_ratio = df.groupby("spk_id")["pause_length speech_length".split()].sum()
df_pause_ratio["pause_ratio"] = (
    df_pause_ratio["pause_length"] / df_pause_ratio["speech_length"]
)
df_pause_ratio["pause_ratio"].hist(bins=30, figsize=(16, 8))
plt.xlabel("Pause ratio")
plt.ylabel("Number of speakers")
plt.title("Pause ratio distribution")
plt.show()

In [ ]:
df_pause_ratio

In [ ]:
# 列ごとのscatter plotを作成する

df_pause_ratio_tmp = df_pause_ratio.rename(
    columns={"pause_length": "pause_length_sum", "speech_length": "speech_length_sum"}
)
sns.pairplot(df_pause_ratio_tmp)

ポーズの長さと発声の長さは少し相関がありそう

- 発話速度を基準として、相対的なポーズの長さをとる。-> 平均発話速度に対して、どれくらいのポーズの長さを
- スタイルごとに、ポーズのプロファイリングや分布をとる。
- 朗読のコーパスで、複数人で、各個人のポーズの取り方を再現する。予測する。
- 